# Finetuning a Pytorch Image Classifier with Ray AIR
In this example we will finetune a pretrained ResNet model with Ray Train. 

For this example, our network architecture consists of the intermediate layer output of a pretrained ResNet model, which feeds into a randomly initialized linear layer that outputs classification logits for our new task.


## Load and preprocess finetuning dataset with Ray Data
This example is adapted from Pytorch's [Fintuning Torchvision Models](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html) tutorial.
We will use *hymenoptera_data* as the finetuning dataset, which contains two classes (bees and ants) and 397 total images (across training and validation). This is a quite small dataset and we use this only for demenstration purpose. 

In [1]:
import argparse
import warnings

warnings.filterwarnings("ignore")


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import numpy as np
from typing import Dict
import random

import ray
from ray.data.datasource.partitioning import Partitioning
from ray.train.torch import TorchCheckpoint


The dataset can be downloaded [here](https://download.pytorch.org/tutorial/hymenoptera_data.zip).

First, we use {meth}`ray.data.read_images <ray.data.read_images>` to load the images. Since the dataset is already structured with directory names as the labels, we can use the {class}`Partitioning <ray.data.datasource.Partitioning>` API to automatically extract image labels.

In [3]:
ray_img_datasets = {}
for split in ["train", "val"]:
    data_folder = f"s3://anonymous@air-example-data-2/hymenoptera_data/{split}"
    partitioning = Partitioning("dir", field_names=["class"], base_dir=data_folder)
    ray_img_datasets[split] = ray.data.read_images(
        data_folder, partitioning=partitioning, mode="RGB"
    )


2023-02-17 12:00:18,772	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.14.21:6379...
2023-02-17 12:00:18,831	INFO worker.py:1553 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
2023-02-17 12:00:19,516	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c72a49158f24c9648dbe5efbee3b2601.zip' (227.47MiB) to Ray cluster...
2023-02-17 12:00:24,343	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c72a49158f24c9648dbe5efbee3b2601.zip'.


We have already load the images from to a Ray dataset, which will partition the whole dataset and distribute the data blocks across the nodes in cluster. You will benefit from faster parallel pre-processing and data ingestion. Notice that the ResNet model was pretrained with hard-coded normalization values. We'll keep these numbers the same for fine-tuning, as shown in *data_transforms*. More details can be found [here](https://pytorch.org/hub/pytorch_vision_resnet/).

In [4]:
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

class_to_idx = {"ants": 0, "bees": 1}


def preprocess(batch: Dict[str, np.ndarray], split: str) -> Dict[str, np.ndarray]:
    transform = data_transforms[split]
    batch["image"] = np.array([transform(img).numpy() for img in batch["image"]])
    batch["label"] = np.array([class_to_idx[cls_name] for cls_name in batch["class"]])
    batch.pop("class")
    return batch


ray_datasets = {
    split: ds.map_batches(
        fn=preprocess, fn_kwargs={"split": split}, batch_format="numpy"
    )
    for split, ds in ray_img_datasets.items()
}


```{note}
Note that **batch** here refers to the chunk of data for preprocessing, not the batch for model training. To learn more about writing functions for {meth}`map_batches <ray.data.Dataset.map_batches>`, read [writing user-defined functions](transform_datasets_writing_udfs).
```

## Initialize Model and Fine-tuning configs

Next let's define our model, you can create a model from a pretrained ResNet, or reload the model checkpoint from a previous run.

In [5]:
def initialize_model():
    # Load pretrained model params
    model = models.resnet50(pretrained=True)

    # Replace the original classifier with a new Linear layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)

    # Ensure all params get updated during fintuning
    for param in model.parameters():
        param.requires_grad = True
    return model


Next, let's define the training configuration, which will be passed into training loop function later.

In [6]:
train_loop_config = {
    "input_size": 224,  # Input image size (224 x 224)
    "batch_size": 32,  # Batch size for training
    "num_epochs": 10,  # Number of epochs to train for
    "lr": 0.001,  # Learning Rate
    "momentum": 0.9,  # SGD optimizer momentum
}


## Define the Training Loop

The `train_loop_per_worker` function defines the finetuning procedure for each worker.

**1. Load dataset shard for each worker**:
- The Trainer will take a dictionary of Ray {class}`~ray.data.Dataset`s as input. These will be preprocessed and accessible in the worker's training loop via {meth}`session.get_dataset_shard() <ray.air.session.get_dataset_shard>`.
- By default, only the dataset under the key "train" will be split into multiple shards. `session.get_dataset_shard()` will return the full dataset for other keys. To configure this, see {class}`~ray.air.DatasetConfig`.
- Use {meth}`iter_torch_batches <ray.data.Dataset.iter_torch_batches>` to iterate the datasets with automatic tensor batching and device placement. If you need a more flexible customized batching function, please refer to our lower-level {meth}`iter_batches <ray.data.Dataset.iter_batches>` API.

**2. Prepare your model**:
- {meth}`train.torch.prepare_model() <ray.train.torch.prepare_model>` will prepares the model for distributed training. Under the hood, it converts your torch model to `DistributedDataParallel` model, which will synchronize its weights across all workers.

**3. Report metrics and checkpoint**:
- {meth}`session.report() <ray.air.session.report>` will report metrics and checkpoints to Ray AIR.
- Saving checkpoints through {meth}`session.report(metrics, checkpoint=...) <ray.air.session.report>` will automatically [upload checkpoints to cloud storage](tune-cloud-checkpointing) (if configured), and allow you to easily enable Ray AIR worker fault tolerance in the future.
- The best checkpoints will be saved according to the specified `checkpoint_score_attribute` in {class}`CheckpointConfig <ray.air.config.CheckpointConfig>`. Here we only save the best model with highest validation accuracy.

In [7]:
import ray.train as train
from ray.air import session
from ray.train.torch import TorchCheckpoint


def evaluate(logits, labels):
    _, preds = torch.max(logits, 1)
    corrects = torch.sum(preds == labels).item()
    return corrects


def train_loop_per_worker(configs):
    warnings.filterwarnings("ignore")
    # Prepare dataloader for each worker
    datasets = dict()
    datasets["train"] = session.get_dataset_shard("train")
    datasets["val"] = session.get_dataset_shard("val")

    # Calculate the batch size for a single worker
    worker_batch_size = configs["batch_size"] // session.get_world_size()

    device = train.torch.get_device()

    # Prepare DDP Model, optimizer, and loss function
    model = initialize_model()  # [TODO]

    model = train.torch.prepare_model(model)

    optimizer = optim.SGD(
        model.parameters(), lr=configs["lr"], momentum=configs["momentum"]
    )
    criterion = nn.CrossEntropyLoss()

    # Start training loops
    for epoch in range(configs["num_epochs"]):
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Create a dataset iterator for the shard on the current worker
            dataset_iterator = datasets[phase].iter_torch_batches(
                batch_size=worker_batch_size, device=device
            )
            for batch in dataset_iterator:
                inputs = batch["image"]
                labels = batch["label"]

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == "train"):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # calculate statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += evaluate(outputs, labels)

            epoch_loss = running_loss / datasets[phase].count()
            epoch_acc = running_corrects / datasets[phase].count()

            if session.get_world_rank() == 0:
                print(
                    "Epoch {}-{} Loss: {:.4f} Acc: {:.4f}".format(
                        epoch, phase, epoch_loss, epoch_acc
                    )
                )

            # Report metrics and checkpoint every epoch
            if phase == "val":
                checkpoint = TorchCheckpoint.from_dict(
                    {
                        "epoch": epoch,
                        "model": model.module.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                    }
                )
                session.report(
                    metrics={"loss": epoch_loss, "acc": epoch_acc},
                    checkpoint=checkpoint,
                )


Next, setup the TorchTrainer:

In [8]:
from moto.server import ThreadedMotoServer
from moto import mock_s3
import boto3
import os
import logging

server = ThreadedMotoServer(port=5002)
server.start()

s3 = boto3.client('s3', endpoint_url='http://localhost:5002')

bucket_name = 'checkpoint-bucket'
s3.create_bucket(Bucket=bucket_name)
logging.getLogger("werkzeug").setLevel(logging.WARNING)


In [9]:
UPLOAD_DIR = "s3://YOUR_BUCKET_NAME"


In [ ]:
UPLOAD_DIR = f"s3://{bucket_name}/results?endpoint_override=http://localhost:5002"


In [11]:
from ray.train.torch import TorchTrainer, TorchCheckpoint
from ray.air.config import ScalingConfig, RunConfig, CheckpointConfig
from ray.tune.syncer import SyncConfig

# Scale out model training across 4 workers, each assigned 1 CPU and 1 GPU.
scaling_config = ScalingConfig(
    num_workers=4, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1}
)

# Save only the latest checkpoint
checkpoint_config = CheckpointConfig(num_to_keep=1)

# Set experiment name and checkpoint configs
run_config = RunConfig(
    name="finetune-resnet",
    sync_config=SyncConfig(upload_dir=UPLOAD_DIR),
    checkpoint_config=checkpoint_config,
)


In [12]:
scaling_config = ScalingConfig(
    num_workers=8, use_gpu=False, resources_per_worker={"CPU": 1}
)
train_loop_config["num_epochs"] = 1


In [13]:
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config=train_loop_config,
    scaling_config=scaling_config,
    run_config=run_config,
    datasets=ray_datasets,
)

result = trainer.fit()


(RayTrainWorker pid=13616, ip=10.0.10.245) 2023-02-17 12:00:53,454	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=13533, ip=10.0.10.245) 2023-02-17 12:00:54,185	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=13533, ip=10.0.10.245) /home/ray/anaconda3/lib/python3.9/site-packages/ray/train/_internal/dataset_iterator.py:64: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset/DatasetPipeline as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(TorchTrainer pid=13533, ip=10.0.10.245)   warnings.warn(
(RayTrainWorker pid=16094, ip=10.0.62.76) 2023-02-17 12:01:00,695	INFO train_loop_utils.py:307 -- Moving model to device: cuda:0
(Ra

(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 0-train Loss: 0.6943 Acc: 0.5082


(RayTrainWorker pid=13075, ip=10.0.5.215) 2023-02-17 12:01:05,988	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)] -> AllToAllOperator[randomize_block_order]
(RayTrainWorker pid=13616, ip=10.0.10.245) 2023-02-17 12:01:05,982	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)] -> AllToAllOperator[randomize_block_order]
(RayTrainWorker pid=82290) 2023-02-17 12:01:05,984	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)] -> AllToAllOperator[randomize_block_order]
(RayTrainWorker pid=16094, ip=10.0.62.76) 2023-02-17 12:01:05,992	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)] -> AllToAllOperator[randomize_block_order]


(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 0-val Loss: 0.5913 Acc: 0.5882


(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 1-train Loss: 0.5521 Acc: 0.7377
(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 1-val Loss: 0.3870 Acc: 0.9150


...

2023-02-17 12:01:42,710	WARNING util.py:244 -- The `process_trial_save` operation took 2.999 s, which may be a performance bottleneck.


(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 8-train Loss: 0.0417 Acc: 1.0000
(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 8-val Loss: 0.1871 Acc: 0.9281


2023-02-17 12:01:46,815	WARNING util.py:244 -- The `process_trial_save` operation took 2.955 s, which may be a performance bottleneck.


(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 9-train Loss: 0.0328 Acc: 1.0000
(RayTrainWorker pid=13616, ip=10.0.10.245) Epoch 9-val Loss: 0.1843 Acc: 0.9281


2023-02-17 12:01:50,955	WARNING util.py:244 -- The `process_trial_save` operation took 2.990 s, which may be a performance bottleneck.
2023-02-17 12:01:54,881	WARNING util.py:244 -- The `process_trial_save` operation took 2.784 s, which may be a performance bottleneck.
2023-02-17 12:01:58,850	INFO tune.py:825 -- Total run time: 72.03 seconds (70.07 seconds for the tuning loop).


Training has finished! The best checkpoint has been saved to the experiment directory, and you can now check the experiment metrics and checkpoint information:

## Load the fine-tuned model for batch prediction

Now, we want to load the trained model and evaluation it on test data.
We can use {ref}`TorchCheckpoint.from_directory() <ray.train.torch.TorchCheckpoint.from_directory>` to load the resulting checkpoint from our fine-tuning run. The {class}`~ray.train.batch_predictor.BatchPredictor` will identify the dict key `"model"` and load the corresponding parameters into the model.
 

The log and checkpoints will be saved into `local_dir` specified in TrainerTrainer. 

For example:
"/tmp/ray_results/finetune-resnet/TorchTrainer_94bb5_00000_0_2023-02-14_14-40-28/checkpoint_000009"

In [15]:
checkpoint_folder = result.checkpoint.uri
print(checkpoint_folder)


file:///tmp/results/finetune-resnet/TorchTrainer_c4b07_00000_0_2023-02-17_12-00-46/checkpoint_000009


In [16]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.torch import TorchCheckpoint, TorchPredictor

ckpt = TorchCheckpoint.from_uri(checkpoint_folder)
predictor = BatchPredictor.from_checkpoint(
    ckpt, TorchPredictor, model=initialize_model()
)


In [17]:
prediction_ds = predictor.predict(
    ray_datasets["val"],
    feature_columns=["image"],
    keep_columns=["label"],
    num_gpus_per_worker=1,
)


2023-02-17 12:02:15,332	INFO batch_predictor.py:184 -- `num_gpus_per_worker` is set for `BatchPreditor`.Automatically enabling GPU prediction for this predictor. To disable set `use_gpu` to `False` in `BatchPredictor.predict`.
2023-02-17 12:02:15,707	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read->MapBatches(preprocess)]
read->MapBatches(preprocess): 100%|██████████| 128/128 [00:06<00:00, 18.66it/s]
2023-02-17 12:02:22,784	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(ScoringWrapper)]
MapBatches(ScoringWrapper), 0 actors [0 locality hits, 1 misses]:   1%|          | 1/128 [00:08<17:51,  8.44s/it]


## Evaluate prediction results

The prediction has finished! We can use `ds.schema()` and `ds.take()` to inspect the data types and record structure.

We can see that there are two keys in the prediction results:
- "predictions": The output logits of our ResNet model, which is a 1000 dimensional tensor.
- "label": The image label. Specified by `keep_columns` in `predictor.predict()`.

In [18]:
prediction_ds.schema()


predictions: extension<arrow.py_extension_type<ArrowTensorType>>
label: int64

In [19]:
prediction_ds.take(1)


[{'predictions': array([ 1.509053 , -1.2755364], dtype=float32), 'label': 0}]

Here we define a function `convert_logits_to_classes` to convert tensor outputs to labels. 

In [20]:
import pandas as pd


def convert_logits_to_classes(batch):
    batch["pred_label"] = np.argmax(batch["predictions"], axis=1)
    batch["correct"] = batch["pred_label"] == batch["label"]
    return batch


predictions = prediction_ds.map_batches(convert_logits_to_classes, batch_format="numpy")
predictions.show(1)

print("Evaluation Accuracy = ", predictions.mean(on="correct"))


2023-02-17 12:02:31,271	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(convert_logits_to_classes)]
MapBatches(convert_logits_to_classes): 100%|██████████| 1/1 [00:00<00:00, 192.68it/s]


{'predictions': array([ 1.509053 , -1.2755364], dtype=float32), 'label': 0, 'pred_label': 0, 'correct': True}


2023-02-17 12:02:31,291	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[aggregate]
Shuffle Reduce: 100%|██████████| 1/1 [00:00<00:00, 123.91it/s]


Evaluation Accuracy =  0.9281045751633987


You can also reuse the evaluation function defined in the training loop by iterating over the dataset. Note that the previous approach using `map_batches()` is more efficient because it parallelizes the evaluation on each partition.

In [21]:
def evaluate(logits, labels):
    _, preds = torch.max(logits, 1)
    corrects = torch.sum(preds == labels).item()
    return corrects


accuracy = 0
for batch in prediction_ds.iter_torch_batches(batch_size=10):
    accuracy += evaluate(batch["predictions"], batch["label"])
accuracy /= prediction_ds.count()

print("Evaluation Accuracy = ", accuracy)


Evaluation Accuracy =  0.9281045751633987


In [ ]:
server.stop()
